# Running ndmg : One Subject

This tutorial provides a basic overview of how to run ndmg manually within Python.
The absolute easiest way is to run the pipeline from the command line once all dependencies are installed using the following command:

`ndmg_bids </absolute/input/dir> </absolute/output/dir>`.

This will run a single session from the input directory, and output the results into your output directory.

### Steps
1. First, we grab some test data and atlases from our github repository. <br>
2. Then, we choose our input parameters (the defaults work fine if you don't want to worry about this!) <br>
3. Last, we run the pipeline.

Running the pipeline is quite simple: call `ndmg_dwi_pipeline.ndmg_dwi_worker` with the correct input flags.

Let's begin!

In [1]:
import os
import os.path as op
import glob
import shutil
from pathlib import Path

from ndmg.scripts import ndmg_dwi_pipeline
from ndmg.utils import s3_utils

/usr/local/Caskroom/miniconda/base/envs/ndmg/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Download test data and atlases

The below code will grab some sample diffusion MRI data from our `neuroparc` repository, as well as the atlases ndmg needs to run. <br>
If you want to explore these data and atlases, you can find it in `~/.ndmg/`.

Note that the below code requires `git lfs` to be installed.

#### Test Data

In [2]:
# Make data directory if it doesn't exist
ndmg_dir = Path.home() / ".ndmg/"
data_dir = ndmg_dir / "data/"

# Remove old data if it exists
if data_dir.is_dir():
    shutil.rmtree(data_dir)
    
# Remove neuroparc if it already exists
if Path("neuroparc").is_dir():
    shutil.rmtree("neuroparc")
    
# Clone the sample data into `~/.ndmg`.
data_dir.mkdir(parents=True)
!git lfs clone https://github.com/neurodata/neuroparc.git
shutil.move("neuroparc/data/BNU1", data_dir)
data_dir = data_dir / "BNU1"

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'neuroparc'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 549 (delta 33), reused 112 (delta 22), pack-reused 417
Receiving objects: 100% (549/549), 6.41 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (88/88), done.


#### Atlases

In [3]:
# Remove old atlas dir if it exists
atlas_dir = ndmg_dir / "ndmg_atlases"
if atlas_dir.is_dir():
    shutil.rmtree(atlas_dir)

# Download atlases to atlas_dir
atlas_dir.mkdir(parents=True)
for name in Path("neuroparc").iterdir():
    shutil.move(str(name), atlas_dir)

In [4]:
# Remove our now-empty directory
shutil.rmtree("neuroparc")

## Choose input parameters

#### Naming Conventions
Here, we define input variables to the pipeline.
To run the `ndmg` pipeline, you need four files:
1. a `t1w` - this is a high-resolution anatomical image.
2. a `dwi` - the diffusion image.
3. bvecs - this is a text file that defines the gradient vectors created by a DWI scan.
4. bvals - this is a text file that defines magnitudes for the gradient vectors created by a DWI scan.

The naming convention is in the [BIDs](https://bids.neuroimaging.io/) spec.

In [5]:
# Specify base directory and paths to input files (dwi, bvecs, bvals, and t1w required)
subject_id = 'sub-0025864'

# Define the location of our input files.
t1w = str(data_dir / f"{subject_id}/ses-1/anat/{subject_id}_ses-1_T1w.nii.gz")
dwi = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.nii.gz")
bvecs = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.bvec")
bvals = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.bval")

print(t1w)
print(dwi)
print(bvecs)
print(bvals)

/Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz
/Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.nii.gz
/Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bvec
/Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bval


#### Parameter Choices
Here, we choose the parameters to run the pipeline with.
If you are inexperienced with diffusion MRI theory, feel free to just use the default parameters.

- *atlases = ['desikan', 'CPAC200', 'DKT', 'HarvardOxfordcort', 'HarvardOxfordsub', 'JHU', 'Schaefer2018-200', 'Talairach', 'aal', 'brodmann', 'glasser', 'yeo-7-liberal', 'yeo-17-liberal']* : The atlas that defines the node location of the graph you create.
- *mod_types = ['det', 'prob']* : Deterministic or probablistic tractography.
- *track_types = ['local', 'particle']* : Local or particle tracking.
- *mods = ['csa', 'csd']* : [Constant Solid Angle](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4360965/) or [Constrained Spherical Deconvolution](https://onlinelibrary.wiley.com/doi/10.1002/ima.22005).
- regs = *['native', 'native_dsn', 'mni']* : Registration style. If native, do all registration in each scan's space; if mni, register scans to the MNI atlas; if native_dsn, do registration in native space, and then fit the streamlines to MNI space.
- vox_size = *['1mm', '2mm']* : Whether our voxels are 1mm or 2mm.
- seeds = int : Seeding density for tractography. More seeds generally results in a better graph, but at a much higher computational cost.

In [6]:
# Use the default parameters.
atlas = 'desikan'
mod_type = 'det'
track_type = 'local'
mod_func = 'csd'
reg_style = 'native'
vox_size = '2mm'
seeds = 20

# Set an output directory
outdir = '/tmp/output_{}_{}_{}_{}_{}_{}_{}'.format(atlas, mod_type, track_type, mod_func, seeds, reg_style, subject_id)
print(f"Your output directory will be : {outdir}")

Your output directory will be : /tmp/output_desikan_det_local_csd_20_native_sub-0025864


#### Get masks and labels
The pipeline needs these two variables as input. <br>
Running the pipeline via `ndmg_bids` does this for you.

In [7]:
# Auto-set paths to neuroparc files
mask = str(atlas_dir / "atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz")
labels = [str(i) for i in (atlas_dir / "atlases/label/Human/").glob(f"*{atlas}*2x2x2.nii.gz")]

print(f"mask location : {mask}")
print(f"atlas location : {labels}")

mask location : /Users/alex/.ndmg/ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz
atlas location : ['/Users/alex/.ndmg/ndmg_atlases/atlases/label/Human/desikan_space-MNI152NLin6_res-2x2x2.nii.gz']


## Run the pipeline!

In [ ]:
ndmg_dwi_pipeline.ndmg_dwi_worker(dwi=dwi, bvals=bvals, bvecs=bvecs, t1w=t1w, atlas=atlas, mask=mask, labels=labels, outdir=outdir, vox_size=vox_size, mod_type=mod_type, track_type=track_type, mod_func=mod_func, seeds=seeds, reg_style=reg_style, clean=False, skipeddy=True, skipreg=True)

dwi = /Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.nii.gz
bvals = /Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bval
bvecs = /Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/dwi/sub-0025864_ses-1_dwi.bvec
t1w = /Users/alex/.ndmg/data/BNU1/sub-0025864/ses-1/anat/sub-0025864_ses-1_T1w.nii.gz
atlas = desikan
mask = /Users/alex/.ndmg/ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz
labels = ['/Users/alex/.ndmg/ndmg_atlases/atlases/label/Human/desikan_space-MNI152NLin6_res-2x2x2.nii.gz']
outdir = /tmp/output_desikan_det_local_csd_20_native_sub-0025864
vox_size = 2mm
mod_type = det
track_type = local
mod_func = csd
seeds = 20
reg_style = native
clean = False
skip eddy = True
skip registration = True
Output directory: /tmp/output_desikan_det_local_csd_20_native_sub-0025864
Adding directory tree...
Connectomes downsampled to given labels: /tmp/output_desikan_det_local_csd_20_native_sub-0025864/dwi/roi-connectomes/desik